In [24]:
# import necessary libraries
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [25]:
# Launching Chrome WebDriver
driver = webdriver.Chrome()
# Navigating to the specified URL
driver.get("https://www.personalinjurylaw.guru/search_results?tid=87&location_value=Arizona&adm_lvl_1_sn=AZ&stateSearchLN=Arizona&country_sn=US&location_type=administrative_area_level_1&stateSearch=AZ&swlat=31.3323448&nelat=37.0037251&swlng=-114.8163229&nelng=-109.0451719&lat=34.0489281&lng=-111.0937311&faddress=Arizona%2C+USA&place_id=ChIJaxhMy-sIK4cRcc3Bf7EnOUI")

In [26]:
# click on the load more button untill it is not available
while True:
    # Wait for 3 seconds before attempting to find the button again
    time.sleep(3)
    try:
        # Try to find the "Load More" button and click it
        btn = driver.find_element("id" ,"btnToLoadMorePost")
        btn.click()
    except NoSuchElementException:
        # If the button is not found, break out of the loop
        break
# Get the page source after loading all posts
doc = driver.page_source

In [32]:
soup = BeautifulSoup(doc, "lxml")
# Function to scrape data from the page using BeautifulSoup
def scrape_page(soup):
    # Extracting names of attorneys
    names = [name.text.strip().replace("- Attorneys", "") for name in soup.find_all('h2', class_='h3')]
    
    # Extracting companies
    containers = soup.find_all('div', class_='mid_section')
    companies = [container.find_all('td')[1].text if len(container.find_all('td')) > 1 else np.nan for container in containers]
    
    # Extracting locations
    location_tag = soup.find_all('i', class_='fa fa-map-marker text-danger')
    locations = [location.find_next_sibling(text=True).strip() for location in location_tag]
    
    # Extracting phone numbers
    phone_numbers = [np.nan if not container.find('div', class_='hide myphoneHide') else container.find('div', class_='hide myphoneHide').text.strip().replace("Call: ", "") for container in containers]

    return names, companies, locations, phone_numbers

In [33]:
# Scrape data from the page
names, companies, locations, phone_numbers = scrape_page(soup)

C:\Users\dell\AppData\Local\Temp\ipykernel_10272\2878756766.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  locations = [location.find_next_sibling(text=True).strip() for location in location_tag]


In [35]:
# Create a dictionary to store the scraped data
data = {
    'Name': names,
    'Company': company,
    'Location': location,
    'Phone Number': phone_numbers
}

# Create a DataFrame
df = pd.DataFrame(data)

In [36]:
# Save the DataFrame to a CSV file
df.to_csv("Arizona-State-Injury-Lawyers.csv")